In [26]:
from astropy.coordinates import EarthLocation, SkyCoord, get_sun, AltAz 
from astropy import time
m1 = SkyCoord.from_name('M1')

#dir() to see attributes

In [27]:
print(m1.frame)

<ICRS Coordinate: (ra, dec) in deg
    (83.63308333, 22.0145)>


In [40]:
sun = get_sun(time.Time.now()).transform_to(AltAz(location=EarthLocation(53.3845258962902, 6.23475766593151)))

In [41]:
print(sun)

<SkyCoord (AltAz: obstime=2021-07-03 11:55:19.431283, location=(3781835.25203023, 5089373.33486723, 688071.22792422) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
    (294.33058188, 38.0766797, 1.52094142e+11)>


In [39]:
sun.alt.deg

39.023916642671054

In [1]:
import sqlite3
import datetime as dt
con = sqlite3.connect('Database.db')

In [3]:
cur = con.cursor()

In [4]:
cur.execute("""CREATE TABLE Schedule
            (object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date, Priority)""")

In [1]:
from Helper_funcs import add_randome_data
import sqlite3
connect = sqlite3.connect('Database.db')
add_randome_data()

INSERT INTO Schedule(object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date, Completed_by, priority) VALUES('M1', 'True', 'Student', '91', '735', '21-06-2020', '30-07-2021', 'None');
INSERT INTO Schedule(object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date, Completed_by, priority) VALUES('M2', 'True', 'OA', '73', '135', '19-05-2020', '21-09-2021', 'None');
INSERT INTO Schedule(object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date, Completed_by, priority) VALUES('M3', 'True', 'Outreach/schools', '80', '1110', '05-01-2021', '23-09-2021', 'None');
INSERT INTO Schedule(object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date, Completed_by, priority) VALUES('M4', 'False', 'OA', '52', '585', '09-06-2020', '15-10-2021', 'None');
INSERT INTO Schedule(object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date, Completed_by, priority) VALUES('M5', 'True', 'OA', '43', '735', '01-04-2021', '0

In [2]:
from Scheduler import daily_Schedule
cont = daily_Schedule('Database.db', 'Schedule')

ipykernel_launcher.py:[range_comp]:254 couldn't add to priority no appropriate range for -1


In [3]:
a = 10
print('| {:20} | {:20} | {:20} | {:20} | {:20} | {:20} | {:20} | {:20} |'.format(*[key for key in cont[0]]))
print('-'*185)
for i in range(len(cont)):
    print('| {:20} | {:20} | {:20} | {:20} | {:20} | {:20} | {:20} | {:20} |'.format(*[cont[i][key] for key in cont[i]]))

| object               | time_sensitive       | Observer_type        | Rarity               | total_length         | Submission_Date      | Completed_by         | priority             |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| M43                  | True                 | Moderator            |                   40 |                  510 | 22-06-2020           | 10-07-2021           |    82.69999999999999 |
| M82                  | False                | Moderator            |                    1 |                  360 | 06-09-2020           | 13-07-2021           |                 75.6 |
| M83                  | True                 | Moderator            |                    2 |                  210 | 09-10-2020           | 06-10-2021           |                 56.6 |
| M13                  | True                 | Moderator            |

In [9]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
m1 = SkyCoord.from_name('M1')
obs = EarthLocation(53.3845258962902, 6.23475766593151) 
observing_time = Time('2010-12-21 1:00')
altaz = AltAz(location=obs, obstime=observing_time)
m1.transform_to(altaz)
print(m1)

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63308333, 22.0145)>


In [31]:
#Equiv to .schema
string = ''
for item in cur.execute('''PRAGMA table_info(observatory_status);'''):
    string += item[1]+', '
print(string)
for row in cur.execute('SELECT * FROM observatory_status'):
        print(row)

date, dome_status, dome_direction, telescope_direction, 
('2021-07-03 15:48:44.347587', 'closed', '[30, 40]', '[50, 60]')


In [5]:
from Helper_funcs import sqlite_get_columns
import sqlite3
connect = sqlite3.connect('Database.db')
{key: None for key in sqlite_get_columns(connect, 'observatory_status').strip('()').split(', ')}

{'date': None,
 'dome_status': None,
 'dome_direction': None,
 'telescope_direction': None}

In [59]:
connect = sqlite3.connect('Database.db')
with connect:
    for item in connect.execute('''PRAGMA table_info({});'''.format('Schedule')).fetchall():
        print(item[1])
    for row in connect.execute('SELECT * FROM {}'.format('Schedule')).fetchall():
        print(row)

object
time_sensitive
Observer_type
Rarity
total_length
Submission_Date


In [12]:
import requests
import json
res = requests.get('https://clearoutside.com/forecast/53.38/6.23')

In [15]:
print(res.text)

<!DOCTYPE html>
<html lang="en">

<head>

  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Clear Outside v1.0 - International Weather Forecasts For Astronomers</title>

  <link rel="shortcut icon" href="https://clearoutside.com/public/images/favicon.png">

  <!-- App Store Link -->

  <meta name="apple-itunes-app" content="app-id=921555752">

  <!-- Bootstrap CSS -->
  <link href="https://clearoutside.com/public/css/bootstrap.min.css?v=03092019a" rel="stylesheet">


  <!-- Clear Outside CSS -->
  <link href="https://clearoutside.com/public/css/clear_outside.css?v=03092019a" rel="stylesheet">


  <!-- Climacons Font CSS -->
  <link href="https://clearoutside.com/public/css/climacons-font.css?v=03092019a" rel="stylesheet">

  <!-- HTML5 Shim and Respond.js IE8 support of HTML5 elements and media queries -->
  <!--[if lt IE 9]>
    <script src="https://oss.maxcdn.com/libs/html5s

In [26]:
text = res.text.strip(' ').split('\n')

In [27]:
print(text)

['<!DOCTYPE html>', '<html lang="en">', '', '<head>', '', '  <meta charset="utf-8">', '  <meta http-equiv="X-UA-Compatible" content="IE=edge">', '  <meta name="viewport" content="width=device-width, initial-scale=1">', '  <title>Clear Outside v1.0 - International Weather Forecasts For Astronomers</title>', '', '  <link rel="shortcut icon" href="https://clearoutside.com/public/images/favicon.png">', '', '  <!-- App Store Link -->', '', '  <meta name="apple-itunes-app" content="app-id=921555752">', '', '  <!-- Bootstrap CSS -->', '  <link href="https://clearoutside.com/public/css/bootstrap.min.css?v=03092019a" rel="stylesheet">', '', '', '  <!-- Clear Outside CSS -->', '  <link href="https://clearoutside.com/public/css/clear_outside.css?v=03092019a" rel="stylesheet">', '', '', '  <!-- Climacons Font CSS -->', '  <link href="https://clearoutside.com/public/css/climacons-font.css?v=03092019a" rel="stylesheet">', '', '  <!-- HTML5 Shim and Respond.js IE8 support of HTML5 elements and media 

In [28]:
conditon = False
count = 0
for i in range(len(text)):
    if '              <ul>              ' == text[i][:32:]:
        conditon = True
        count += 1
    elif condition:
        sub_ls.append(text[i])
    elif '              </ul>'  == text[i][:19:]:
        condition = False
        


        


SyntaxError: invalid syntax (<ipython-input-28-4298a5e3f36a>, line 2)

In [33]:
len('              </ul>')

19